In [1]:
#必要なモジュールをインポート
import pandas as pd
import numpy as np
#可視化
import matplotlib.pyplot as plt
import seaborn as sns
#単回帰分析
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
#ロジスティック回帰
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score,roc_auc_score,roc_curve, auc

import datetime as dt
from mpl_toolkits.mplot3d import Axes3D  #3Dplot
from sklearn import preprocessing #標準化

from datetime import datetime
# 標準化
import scipy
#可視化
import matplotlib.pyplot as plt
# k-means
from sklearn.cluster import KMeans
import os
import xlrd
import numpy as np
import pandas as pd
#import dask.dataframe as dd
from os import listdir
from tqdm import tqdm_notebook as tqdm
import warnings

In [ ]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
os.getcwd()

## 各データ読み込み

In [13]:
w1 = pd.read_csv('C:\\Users\\taban\\Desktop\\3年講義\\秋学期\\マネジメント演習\\本データ\\気象データ\\つくば市気象データ2017.9-2018.8.csv',encoding = "shift_jis")
w2 = pd.read_csv('C:\\Users\\taban\\Desktop\\3年講義\\秋学期\\マネジメント演習\\本データ\\気象データ\\つくば市気象データ2018.9-2019.8.csv',encoding = "shift_jis")
w3 = pd.read_csv('C:\\Users\\taban\\Desktop\\3年講義\\秋学期\\マネジメント演習\\本データ\\気象データ\\つくば市気象データ2019.9-2020.8.csv',encoding = "shift_jis")

In [14]:
w1 = w1.dropna()
w2 = w2.dropna()
w3 = w3.dropna()

w1 = w1.drop(w1.index[0])
w2 = w2.drop(w2.index[0])
w3 = w3.drop(w3.index[0])

w1 = w1.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1)
w2 = w2.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1)
w3 = w3.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1)

w1.columns = ['日時','天気'] 
w2.columns = ['日時','天気'] 
w3.columns = ['日時','天気'] 

weather = pd.concat([w1,w2,w3], axis=0)

In [16]:
#今回使用するリストはこれ
column_list = ['取引時間', '取引連番','ノードNO']
df = df_tempo[column_list]

In [17]:
# 取引時間から時間だけを取り出しておく
df['取引時間'] = pd.to_datetime(df['取引時間'])
#df['取引時間'] = df['取引時間'].dt.strftime('%Y%m%d%H%M').astype(int)
#曜日ダミー出す用
df1=df[['取引連番','取引時間','ノードNO']]
#それ以外を取引連番毎に結合
df2=df.groupby('取引連番').agg({'ノードNO':np.mean,'取引時間':np.max})

In [18]:
#20日（Tポイント1.5倍）でダミー
dm_20 = pd.get_dummies(df1['取引時間'].dt.strftime('%d') == '20')
df1['20日'] = dm_20[True]

#水曜なら１の水曜ダミーと木曜なら１の木曜日ダミーを作成する
dm_m = pd.get_dummies(df1['取引時間'].dt.strftime('%a') == 'Mon')
df1['Mon'] = dm_m[True]

dm_w = pd.get_dummies(df1['取引時間'].dt.strftime('%a') == 'Wed')
df1['Wed'] = dm_w[True]
dm_Th = pd.get_dummies(df1['取引時間'].dt.strftime('%a') == 'Thu')
df1['Thu'] = dm_Th[True]

#取引連番毎に結合
newdf1 = df1.groupby('取引連番').agg(np.mean)

#列名変える
newdummy = newdf1

In [19]:
#こっからレジ混雑の話
df_regi_1 = df2[df2['ノードNO'] == 1] #レジ1のみ抽出
df_regi_101 = df2[df2['ノードNO'] == 101] #レジ101のみ抽出
df_regi_102 = df2[df2['ノードNO'] == 102] #レジ102のみ抽出
df_regi_103 = df2[df2['ノードNO'] == 103] #レジ103のみ抽出
df_regi_104 = df2[df2['ノードNO'] == 104] #レジ104のみ抽出
df_regi_105 = df2[df2['ノードNO'] == 105] #レジ104のみ抽出

In [20]:
#日付ごとの引き算して結合（それぞれで）
diff_1=df_regi_1['取引時間'].diff()
regi_1 = pd.concat([df_regi_1, diff_1], axis=1)
regi_1.columns = ['ノードNO','取引時間','経過時間']
#後々「取引を行ったレジ数」を計算する時のために、ノードNOのダミー変数を作る
regi_1['1']=1

#diffは日付ごとの引き算
diff_101=df_regi_101['取引時間'].diff()
regi_101 = pd.concat([df_regi_101, diff_101], axis=1)
regi_101.columns = ['ノードNO','取引時間','経過時間']
regi_101['101']=1

diff_102=df_regi_102['取引時間'].diff()
regi_102 = pd.concat([df_regi_102, diff_102], axis=1)
regi_102.columns =  ['ノードNO','取引時間','経過時間']
regi_102['102']=1

diff_103=df_regi_103['取引時間'].diff()
regi_103 = pd.concat([df_regi_103, diff_103], axis=1)
regi_103.columns = ['ノードNO','取引時間','経過時間']
regi_103['103']=1

diff_104=df_regi_104['取引時間'].diff()
regi_104 = pd.concat([df_regi_104, diff_104], axis=1)
regi_104.columns =  ['ノードNO','取引時間','経過時間']
regi_104['104']=1

diff_105=df_regi_105['取引時間'].diff()
regi_105 = pd.concat([df_regi_105, diff_105], axis=1)
regi_105.columns =  ['ノードNO','取引時間','経過時間']
regi_105['105']=1

In [21]:
#全部合わせる
regi_all = pd.concat([regi_1, regi_101, regi_102, regi_103, regi_104, regi_105], axis=0)
train_all = pd.concat([regi_all,newdummy], axis=1)

In [22]:
#混雑（前後の時間差）が分かればよいため各レジ最初の決済（欠損値）削除
train_all=train_all.dropna(subset=['経過時間'])
#経過時間を秒数に変える
train_all['経過時間']=train_all['経過時間'].dt.seconds

## 本作業

In [23]:
a=train_all #名前　a　に変更

In [24]:
#2分ずつ並んだら混雑とみなすためダミー化　各レジで1がどれだけ連続しているかをこれから見る
a['経過時間'] = pd.get_dummies(a['経過時間']>120)

In [26]:
c = a.groupby(pd.Grouper(key='取引時間',freq='10min')).agg({'経過時間':np.sum,'20日':np.mean,'Mon':np.mean,'Wed':np.mean,'Thu':np.mean,'1':np.mean,'101':np.mean,'102':np.mean,'103':np.mean,'104':np.mean,'105':np.mean}).reset_index()

In [27]:
c=c.dropna(subset=['20日'])
c=c.fillna(0)
c['取引有無'] = pd.Series(c['1']+c['101']+c['102']+c['103']+c['104']+c['105'], index=c.index )
c['混雑有無']=0
c.loc[c['経過時間'] >= 3, '混雑有無'] = 1

In [28]:
c['year']=  c['取引時間'].dt.strftime('%Y').astype(int)
c['month']= c['取引時間'].dt.strftime('%m').astype(int)
c['day']= c['取引時間'].dt.strftime('%d').astype(int)
c['hour']= c['取引時間'].dt.strftime('%H').astype(int)
c['min']= c['取引時間'].dt.strftime('%M').astype(int)

In [31]:
weather['日時'] = pd.to_datetime(weather['日時'])
weather['year']=  weather['日時'].dt.strftime('%Y').astype(int)
weather['month']= weather['日時'].dt.strftime('%m').astype(int)
weather['day']= weather['日時'].dt.strftime('%d').astype(int)
weather['hour']= weather['日時'].dt.strftime('%H').astype(int)
#天気が車使うときか否かでダミー化(9-19)
weather['天気']=weather['天気'].astype(int)
weather.loc[weather['天気']<9,['天気']]=0
weather.loc[weather['天気']>19,['天気']]=0
weather.loc[weather['天気']>=9,['天気']]=1
weather = weather[['year','month','day','hour','天気']]
#weather

9霧雨
10雨
11みぞれ
12雪
13あられ
14ひょう
15雷
16しゅう雨止み間の雨
17着氷性の雨
18着氷性の霧雨
19しゅう雪止み間の雪
の時は車だから交通手段が別(1とする)

In [32]:
#共通列を持つものでweatherとsakura_dataをまとめる
sakura_all = pd.merge(c, weather, on=['year', 'month','day','hour'])
#sakura_all.head(300)

In [33]:
#print(sakura_all.isnull().any())

ちなみにsakura_allとsakura_dataで行数違うのは2020年8月分があるかないか

In [36]:
d2 = sakura_all.drop(['1','101','102','103','104','105','経過時間'], axis=1)
#d2.head(300)
#「最適開放レジ数」の列を追加
d2['最適開放レジ数'] = pd.Series(d2['混雑有無']+d2['取引有無'], index=d2.index )
#列名の変更
d2.columns = ['取引時間','20日','Mon','Wed','Thu','開放レジ数','混雑したレジの有無','year','month','day','hour','min','天気','最適開放レジ数']
#print(d2.isnull().any())

In [38]:
d2 = d2.groupby(pd.Grouper(key='取引時間',freq='1h')).agg({'20日':np.mean,'Mon':np.mean,'Wed':np.mean,'Thu':np.mean,'hour':np.mean,'天気':np.mean,'最適開放レジ数':np.max}).reset_index()
d2=d2.dropna(subset=['20日'])
d2['20日'] = pd.Series(round(d2['20日']), index=d2.index )
d2['Mon'] = pd.Series(round(d2['Mon']), index=d2.index )
d2['Wed'] = pd.Series(round(d2['Wed']), index=d2.index )
d2['Thu'] = pd.Series(round(d2['Thu']), index=d2.index )

## trainデータフレーム作成

In [219]:
e = d2[(d2['取引時間'] > dt.datetime(2017,9,1)) & (d2['取引時間'] < dt.datetime(2020,8,31))]
train = e[['hour','20日','Mon','Wed','Thu','天気','最適開放レジ数']]

## testデータフレーム作成

In [222]:
f = d2[(d2['取引時間'] >= dt.datetime(2020,8,31)) & (d2['取引時間'] < dt.datetime(2020,9,1))]
test = f[['hour','20日','Mon','Wed','Thu','天気','最適開放レジ数']]

## 学習準備

In [225]:
#機械学習
#予測モデルの作成
y_train=train[['最適開放レジ数']]
x_train=train[['hour','20日','Mon','Wed','Thu','天気',]]    

In [226]:
#正規化
sscaler = preprocessing.StandardScaler()
sscaler.fit(x_train)
xss_sk = sscaler.transform(x_train) 
sscaler.fit(y_train)
yss_sk = sscaler.transform(y_train)

#print(xss_sk)
#print(yss_sk)

In [227]:
#予測
y_test=test[['最適開放レジ数']]
x_test=test[['hour','20日','Mon','Wed','Thu','天気']]

## light GBM

In [51]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)

# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [59]:
# モデルの学習
model = lgb.LGBMClassifier() # モデルのインスタンスの作成
model.fit(x_train, y_train) # モデルの学習

# テストデータの予測クラス (予測クラス(0 or 1 or 2)を返す)
y_pred = model.predict(x_test)
# テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す)
y_pred_prob = model.predict_proba(x_test)

In [ ]:
# モデル評価
# acc : 正答率

acc = accuracy_score(y_test,y_pred)
print('Acc :', acc)

## ランダムフォレスト（こちらを選択）

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
# 識別モデルの構築
random_forest = RandomForestClassifier(max_depth=30, n_estimators=30, random_state=42)
random_forest.fit(x_train, y_train)

# 予測値算出
y_pred = random_forest.predict(x_test)

#モデルを作成する段階でのモデルの識別精度
trainaccuracy_random_forest = random_forest.score(x_train, y_train)
print('TrainAccuracy: {}'.format(trainaccuracy_random_forest))

#作成したモデルに学習に使用していない評価用のデータセットを入力し精度を確認
accuracy_random_forest = accuracy_score(y_test, y_pred)
print('Accuracy: {}'.format(accuracy_random_forest))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#confusion matrix
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, square=True, annot=True, cbar=False, fmt='d', cmap='RdPu')
plt.xlabel('predicted class')
plt.ylabel('true value')

In [ ]:
# 変数の重要度を可視化
importance = pd.DataFrame({ '変数' :x_train.columns, '重要度' :random_forest.feature_importances_})
importance

## 結果の可視化

In [ ]:
output=test
output['最適開放レジ数予測'] = y_pred
output

In [ ]:
plt.figure(figsize=(16, 8), dpi=100)
plt.plot(output['hour'], output['最適開放レジ数'], marker="o")
plt.title('The hourly optimal number of cashiers')
plt.xlabel('hour')
plt.ylabel('the opitimal number of cashiers')
plt.show()

In [ ]:
plt.figure(figsize=(16, 8), dpi=100)
plt.plot(output['hour'], output['最適開放レジ数予測'], marker="o")
plt.title('precision(The hourly optimal number of cashiers)')
plt.xlabel('hour')
plt.ylabel('the opitimal number of cashiers')
plt.show()

## tensorflowの回帰問題

こちらや重回帰などは今回多クラス分類のため選択せず

## tensorflow多クラス分類問題（softmax）

In [ ]:
# TensorFlow と tf.keras のインポート
import tensorflow as tf
from tensorflow import keras

# ヘルパーライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

import kerastuner as kt

print(tf.__version__)

In [ ]:
from tensorflow.keras.utils import to_categorical

# one-hotエンコーディング
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train

In [ ]:
y_test

In [282]:
model = keras.Sequential([
    keras.layers.Input(len(x_train.keys())),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(7, activation='softmax')#y_trainが7列だから
])

## 3cellハイパーチューニング

In [283]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=50, verbose=1)

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
np.argmax(y_pred, axis=1)

In [ ]:
output=test
output['最適開放レジ数予測'] = np.argmax(y_pred, axis=1)
output

In [ ]:
plt.figure(figsize=(16, 8), dpi=100)
plt.plot(output['hour'], output['最適開放レジ数予測'], marker="o")
plt.title('precision(The hourly optimal number of cashiers)')
plt.xlabel('hour')
plt.ylabel('the opitimal number of cashiers')
plt.show()